In [ ]:
import numpy as np
import pandas as pd

# pd.set_option('display.height', 1000)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.set_option("max_colwidth", 200)
from IPython.core.display import HTML, display

display(HTML("<style>.container { width:100% !important; }</style>"))


def df_stats(df):
    from tabulate import tabulate

    print("\n***** Shape: ", df.shape, " *****\n")

    columns_list = df.columns.values.tolist()
    isnull_list = df.isnull().sum().values.tolist()
    isunique_list = df.nunique().values.tolist()
    dtypes_list = df.dtypes.tolist()

    list_stat_val = list(zip(columns_list, isnull_list, isunique_list, dtypes_list))
    df_stat_val = pd.DataFrame(list_stat_val, columns=["Name", "Null", "Unique", "Dtypes"])
    print(tabulate(df_stat_val, headers="keys", tablefmt="psql"))
    return df.head()

In [ ]:
import pickle

# Load the pickle file containing (all_reports, all_text_embeddings)
with open("text_embeddings.pkl", "rb") as f:
    all_reports, all_text_embeddings = pickle.load(f)

# Print the first 5 indices and their corresponding reports
for i in range(5):
    print(f"Index {i}: {all_reports[i]}")

In [ ]:
print(len(all_reports))

In [ ]:
import pickle

import pandas as pd

# Specify the epoch number you want to inspect
epoch = 10  # Change this to the desired epoch number

# Paths to the CSV and pickle files
output_dir = "outputs"  # Change if your outputs are in a different directory
val_csv_path = f"{output_dir}/val_epoch{epoch}.csv"
pkl_path = f"{output_dir}/text_embeddings.pkl"

# Load the CSV containing predictions
df = pd.read_csv(val_csv_path)

# Load the pickle containing (all_reports, all_text_embeddings)
with open(pkl_path, "rb") as f:
    all_reports, all_text_embeddings = pickle.load(f)

# Display top 5 predictions for each video
for idx, row in df.iterrows():
    file_name = row["FileName"]
    ground_truth_idx = row["ground_truth_idx"]
    predicted_indices = [row[f"predicted_idx_{i}"] for i in range(1, 6)]

    # Retrieve the corresponding text reports
    ground_truth_report = (
        all_reports[ground_truth_idx] if 0 <= ground_truth_idx < len(all_reports) else "Unknown"
    )
    predicted_reports = []
    for p_idx in predicted_indices:
        if 0 <= p_idx < len(all_reports):
            predicted_reports.append(all_reports[p_idx])
        else:
            predicted_reports.append("Unknown")

    print(f"FileName: {file_name}")
    print(f"Ground Truth Index: {ground_truth_idx}")
    print(f"Ground Truth Report: {ground_truth_report}")
    print("Top 5 Predicted Reports:")
    for i, report_text in enumerate(predicted_reports, start=1):
        display(f"  {i}. {report_text}")
    display("-" * 50)

In [ ]:
### Stenting procedure